In [2]:
'''
管道+模型选择
'''
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName("PySpark_DataFrame").master("local[2]").\
config("spark.sql.warehouse.dir","file:///E:/input/spark/warehouse").getOrCreate()

In [12]:
from pyspark.ml.regression import LinearRegression,GeneralizedLinearRegression

# Load training data
dataset = spark.read.format("libsvm").load("file:///E://download/sample_libsvm_data.txt")

glr = GeneralizedLinearRegression(family="gaussian", link="identity", maxIter=10, regParam=0.3)
lr = LinearRegression(maxIter=10)

#%debug
# Fit the model
#model = lr.fit(dataset)
model = glr.fit(dataset)

# Print the coefficients and intercept for generalized linear regression model
#print("Coefficients: " + str(model.coefficients))
print("Intercept: " + str(model.intercept))

# Summarize the model over the training set and print out some metrics
summary = model.summary
#print("Coefficient Standard Errors: " + str(summary.coefficientStandardErrors))
#print("T Values: " + str(summary.tValues))
#print("P Values: " + str(summary.pValues))
#print("Dispersion: " + str(summary.dispersion))
#print("Null Deviance: " + str(summary.nullDeviance))
#print("Residual Degree Of Freedom Null: " + str(summary.residualDegreeOfFreedomNull))
print("Deviance: " + str(summary.deviance))
#print("Residual Degree Of Freedom: " + str(summary.residualDegreeOfFreedom))
print("AIC: " + str(summary.aic))
#print("Deviance Residuals: ")
summary.residuals().show()

Intercept: 0.6139552540513642
Deviance: 0.21531039194374035
AIC: 1057.7032271916974
+--------------------+
|   devianceResiduals|
+--------------------+
|-0.04489581502374462|
|0.010287260575441826|
|-0.02861493991522...|
| 0.06442964489573966|
|0.007493190261459337|
|-0.07783803568944425|
|0.009249931839144132|
| 0.06668312531069676|
|-0.08645681324247279|
|-0.01224597666755789|
|0.001166573732491516|
| 0.01765209729353323|
|  0.0199905433514197|
| 0.03588518195566892|
|-0.00999139248883607|
| 0.01944938727657186|
|0.010817641622237972|
| 0.10290220859513355|
|0.021070016734725194|
|  7.6253426693762E-4|
+--------------------+
only showing top 20 rows



In [9]:
len(summary.residuals().collect())

100

In [36]:
from sklearn.metrics import r2_score
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder,TrainValidationSplit,CrossValidator

lr = LinearRegression(maxIter=10)
#glr = GeneralizedLinearRegression(family="gaussian", link="identity", maxIter=10, regParam=0.3)
glr = GeneralizedLinearRegression(family="tweedie",maxIter=10, regParam=0.3)
#paramGrid = ParamGridBuilder().addGrid(glr.family,["tweedie"]).addGrid(glr.link,["identity","log"]).build()
paramGrid = ParamGridBuilder().addGrid(glr.variancePower,[0,1,1.5,2,3]).build()

#model3 = lr.fit(dataset)
#preds = model2.transform(dataset)
#eva = RegressionEvaluator(labelCol="label",predictionCol='prediction',metricName='rmse')
#rmse = eva.evaluate(preds)
#print(rmse)
#r2_score(dataset.label,pred.prediction)
regEvaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")

tvs = TrainValidationSplit(estimator=glr,estimatorParamMaps=paramGrid,evaluator=regEvaluator,trainRatio=0.7)

tvs_model = tvs.fit(dataset)
# Print the coefficients and intercept for generalized linear regression model
#print("Coefficients: " + str(model2.coefficients))
#print("Intercept: " + str(model2.intercept))

Py4JJavaError: An error occurred while calling o3434.w.
: java.lang.ClassCastException


<bound method DataFrame.printSchema of DataFrame[label: double, features: vector]>